## Training GAT network for finding bike traffic patterns

In [1]:
import mlflow

# Specify tracking server
mlflow.set_tracking_uri(uri="http://localhost:5000")
mlflow.set_experiment("Predict bike traffic_GAT_fixed_floor_area")

<Experiment: artifact_location='mlflow-artifacts:/797981034638231270', creation_time=1725170517251, experiment_id='797981034638231270', last_update_time=1725170517251, lifecycle_stage='active', name='Predict bike traffic_GAT_fixed_floor_area', tags={}>

In [16]:
import os

os.environ['USE_PYGEOS'] = '0'
import datetime
from enum import Enum
from pathlib import Path

import geopandas as gpd
import momepy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import networkx as nx
import numpy as np
# import setuptools
import torch
from matplotlib.animation import FuncAnimation
from shapely import LineString
# from tqdm import tqdm
from tqdm.notebook import tqdm
from torch_geometric.utils.convert import from_networkx
# from torch_geometric.transforms import NormalizeScale

from utils.config import SIGNATURE


class OptimizerType(Enum):
    ADAM = {
        "learning_rate": 0.005,
        "betas": (0.9, 0.999),
        "eps": 1e-08,
        "weight_decay": 0.001,
    }
    SGD = {
        "learning_rate": 0.0001,
        "momentum": 0.8,
        "weight_decay": 0.0001,
    }


params = {
    "train_ratio": 0.8,
    "num_hidden_channels": 24,
    "GNN_dropout_rate": 0.2,
    "head_num": 12,
    "optimizer": OptimizerType.ADAM,  # or OptimizerType.SGD
    "num_layers": 3,
    "batch_size": 128,
    #  the learning rate of each parameter group using a cosine annealing schedule
    "scheduler": False
    # "scheduler_gamma": [0.8],
    # "pos_weight": [1.3],
    # "model_embedding_size": [64],
    # "model_attention_heads": [3],
    # "model_layers": [4],
    # "model_dropout_rate": [0.2],
    # "model_top_k_ratio": [0.5],
    # "model_top_k_every_n": [1],
    # "model_dense_neurons": [256]
}

In [17]:
prefix_name = "testF4mean"
place_name = "Dresden"
file_name = place_name + "_" + prefix_name

len_normalize = 100

data_path = Path().resolve() / "data" / "processed" / f"{file_name}.gpkg"
traffic = gpd.read_file(data_path, driver='GPKG', layer='traffic')
print(f"loading data from {data_path}")

# traffic.crs = "epsg:4326"
traffic = traffic.to_crs(4839)
# traffic.crs = "epsg:3857"

loaded_graph = momepy.gdf_to_nx(traffic, approach="primal")
nodes = max(nx.connected_components(loaded_graph), key=len)
connected_graph = nx.subgraph(loaded_graph, nodes)
connected_graph = nx.Graph(connected_graph)
# H_nodes, H_edges = momepy.nx_to_gdf(H)
# print(H_edges.head(12))

num_idx_graph = nx.convert_node_labels_to_integers(
    connected_graph, label_attribute="coordinate")
node_loc = {
    i: num_idx_graph.nodes[i]["coordinate"]
    for i in range(num_idx_graph.number_of_nodes())
}

lineGraph = nx.line_graph(num_idx_graph)
# for node in lineGraph.copy():
#     lineGraph.add_node(node,mm_len=num_idx_graph.edges[node]["mm_len"]/len_normalize,
#                 occurrence=num_idx_graph.edges[node]["occurrence"],
#                 bikeability=num_idx_graph.edges[node]["bikeability"]
#                 )

attr_list = [
    "mm_len", "occurrence", "bikeability", "floor_area", "department_store",
    "supermarket", "kiosk", "variety_store", "foodstore", "bakery",
    "ice_cream", "bookstore", "ticket", "copyshop", "fashion", "DIY",
    "houseware", "furniture", "electronics", "sportstore", "florist",
    "laundry", "petstore", "toystore", "cafe", "restaurant", "pub", "theatre",
    "cinema", "market", "place_of_worship", "bank", "pharmacy", "chemist",
    "post_office", "townhall", "library", "kindergarten", "school", "college",
    "park", "stadium", "sportplace", "university"
]
attr_list_no_occurrence = [
    "mm_len", "bikeability", "floor_area", "department_store", "supermarket",
    "kiosk", "variety_store", "foodstore", "bakery", "ice_cream", "bookstore",
    "ticket", "copyshop", "fashion", "DIY", "houseware", "furniture",
    "electronics", "sportstore", "florist", "laundry", "petstore", "toystore",
    "cafe", "restaurant", "pub", "theatre", "cinema", "market",
    "place_of_worship", "bank", "pharmacy", "chemist", "post_office",
    "townhall", "library", "kindergarten", "school", "college", "park",
    "stadium", "sportplace", "university"
]
for node in lineGraph.copy():
    attr_dict = {
        attr_name: num_idx_graph.edges[node][attr_name]
        for attr_name in attr_list
    }
    lineGraph.add_node(node, **attr_dict)

data = from_networkx(lineGraph, group_node_attrs=attr_list_no_occurrence)
data.x[:, 0] = data.x[:, 0] / len_normalize
# data.x = F.normalize(data.x, dim=0)
# data.x = z_score(data.x)
data.y = data.occurrence.unsqueeze(1)
del data.occurrence
# data = transform(data)
# type(pyg_graph)
print(data)
# print(nx.number_of_edges(lineGraph))
# L2.edges
# print(pyg_graph.num_nodes)


loading data from E:\MyDoc\Deep_Thinkvitality\MLcode\Bike_Prediction\data\processed\Dresden_testF4mean.gpkg
Data(edge_index=[2, 55050], x=[12912, 43], y=[12912, 1])


In [4]:
# Draw the traffic flow in designated areas

# _, gdf_edges = momepy.nx_to_gdf(connected_graph)
# fig, ax = plt.subplots(1, 1, figsize=(6, 6))
# gdf_edges.plot(column="occurrence",
#                ax=ax,
#                legend=True,
#                legend_kwds={"shrink": 0.5},
#                norm=colors.LogNorm(vmin=1, vmax=gdf_edges["occurrence"].max()))
# ax.set_axis_off()

# fig_path = Path().resolve() / "data" / "image"
# fig_path.mkdir(parents=True, exist_ok=True)
# plt.savefig(fig_path / f"{file_name}.svg", format="svg")


In [5]:
# find histogram of features
# idx = 0
# test = data.x[:,idx]
# plt.hist(test,bins=20,range=(0, 5))
# print(attr_list_no_occurrence[idx])
# print(max(test))


In [6]:
# gdf_edges.explore()

Split the data to trains and test sets

In [18]:
now = datetime.datetime.now()
time_tag = now.strftime("%m%d_%H%M")

data.train_mask = torch.rand(data.num_nodes) < params["train_ratio"]
data.test_mask = ~data.train_mask
mask_path = Path().resolve() / "data" / "mask"
mask_path.mkdir(parents=True, exist_ok=True)
np.save(mask_path / f"{file_name}_{time_tag}.npy", data.train_mask)

In [19]:
from torch.nn import Linear, MSELoss
import torch.nn.functional as F
# from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv, TopKPooling


# Define the GAT model
class GAT(torch.nn.Module):

    def __init__(self, num_features, num_layers, hidden_channels, heads,
                 dropout):
        super(GAT, self).__init__()

        self.dropout = dropout
        # GNN layers
        self.convs = torch.nn.ModuleList()
        self.convs.append(GATConv(num_features, hidden_channels,
                                  heads))  # First layer

        for _ in range(1, num_layers - 1):  # Middle layers
            self.convs.append(
                GATConv(hidden_channels * heads, hidden_channels, heads))

        self.convs.append(
            GATConv(hidden_channels * heads,
                    hidden_channels,
                    heads=1,
                    concat=False))  # Last layer

        # linear layers
        self.linear1 = Linear(hidden_channels, 24)
        self.linear2 = Linear(24, 24)
        self.linear3 = Linear(24, 1)

    def forward(self, x, edge_index):
        # Iterate over GNN layers
        for _, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        # linear layer
        x = self.linear1(x)
        x = F.elu(x)
        x = self.linear2(x)
        x = F.elu(x)
        x = self.linear3(x)
        return x


model = GAT(data.num_features, params["num_layers"],
            params["num_hidden_channels"], params["head_num"],
            params["GNN_dropout_rate"])
print(model)
# print(f"Number of parameters: {count_parameters(model)}")

GAT(
  (convs): ModuleList(
    (0): GATConv(43, 24, heads=12)
    (1): GATConv(288, 24, heads=12)
    (2): GATConv(288, 24, heads=1)
  )
  (linear1): Linear(in_features=24, out_features=24, bias=True)
  (linear2): Linear(in_features=24, out_features=24, bias=True)
  (linear3): Linear(in_features=24, out_features=1, bias=True)
)


In [20]:
from torch_geometric.loader import ClusterData, ClusterLoader

torch.manual_seed(12345)
cluster_data = ClusterData(data, num_parts=512)  # 1. Create subgraphs. 128
train_loader = ClusterLoader(
    cluster_data, batch_size=params["batch_size"],
    shuffle=True)  # 2. Stochastic partioning scheme. 32

print()
total_num_nodes = 0
for step, sub_data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of nodes in the current batch: {sub_data.num_nodes}')
    print(sub_data)
    print()
    total_num_nodes += sub_data.num_nodes

print(f'Iterated over {total_num_nodes} of {data.num_nodes} nodes!')

Computing METIS partitioning...



Step 1:
Number of nodes in the current batch: 3222
Data(x=[3222, 43], y=[3222, 1], train_mask=[3222], test_mask=[3222], edge_index=[2, 9940])

Step 2:
Number of nodes in the current batch: 3230
Data(x=[3230, 43], y=[3230, 1], train_mask=[3230], test_mask=[3230], edge_index=[2, 10042])

Step 3:
Number of nodes in the current batch: 3225
Data(x=[3225, 43], y=[3225, 1], train_mask=[3225], test_mask=[3225], edge_index=[2, 9480])

Step 4:
Number of nodes in the current batch: 3235
Data(x=[3235, 43], y=[3235, 1], train_mask=[3235], test_mask=[3235], edge_index=[2, 9418])

Iterated over 12912 of 12912 nodes!


Done!


In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def train():
    model.train()
    running_loss = 0.0
    step = 0

    for sub_data in train_loader:  # Iterate over each mini-batch.

        sub_data = sub_data.to(device)
        optimizer.zero_grad()
        out = model(sub_data.x, sub_data.edge_index)

        # Only use nodes with labels available for loss calculation --> mask
        loss = criterion(out[sub_data.train_mask],
                         sub_data.y[sub_data.train_mask])
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        step += 1

    return running_loss / step


def test():
    model.eval()
    out = model(data.x, data.edge_index)
    loss = criterion(out[data.test_mask], data.y[data.test_mask])
    return loss


# Function to create a frame for the animation
def create_frame(epoch):
    plt.figure(figsize=(6, 6))

    prediction = model(data.x, data.edge_index)
    truth = data.y.detach().numpy()
    prediction = prediction.detach().numpy()
    pair_predict = np.concatenate((truth, prediction), axis=1)

    max_ground_value = max(data.y.detach().numpy())
    mask = data.train_mask.numpy()

    plt.plot(pair_predict[mask, 0],
             pair_predict[mask, 1],
             'o',
             color='royalblue',
             markersize=2,
             label='train data')
    plt.plot(pair_predict[~mask, 0],
             pair_predict[~mask, 1],
             'o',
             color='indianred',
             markersize=2,
             label='test data')

    plt.text(0.05,
             0.95,
             f'Epoch: {epoch:04d}',
             transform=plt.gca().transAxes,
             fontsize='medium',
             verticalalignment='top')
    plt.xlabel("Ground Truth")
    plt.ylabel("Prediction")
    plt.xlim([0, max_ground_value])
    plt.ylim([0, max_ground_value])
    plt.legend(loc='upper left')

    return plt.gca().figure,


# Create the animation
def create_animation(max_epoch):
    fig, ax = plt.subplots(figsize=(6, 6))
    ani = FuncAnimation(fig,
                        create_frame,
                        frames=range(max_epoch),
                        repeat=False)

    # Save the animation
    # animation_path = output_path / f"{file_name}_{time_tag}.mp4"
    animation_path = Path().resolve() / "data" / "anima"
    animation_path.mkdir(parents=True, exist_ok=True)
    ani.save(animation_path / f"{file_name}_{time_tag}.gif",
             writer='ffmpeg',
             fps=5)

    plt.close(fig)


def animation():
    animation_count = 0
    max_groud_value = max(data.y.detach().numpy())
    mask = data.train_mask.numpy()
    prediction = model(data.x, data.edge_index)
    truth = data.y.detach().numpy()
    prediction = prediction.detach().numpy()
    pair_predict = np.concatenate((truth, prediction), axis=1)

    animation_path = Path().resolve()/"data"/"anima"/\
            f"{file_name}_{time_tag}_{animation_count:03d}.png"

    scatter1 = plt.plot(pair_predict[mask.transpose().ravel(), 0],
                        pair_predict[mask.transpose().ravel(), 1],
                        'o',
                        color='royalblue',
                        markersize=2,
                        label='train data')
    scateer2 = plt.plot(pair_predict[~mask.transpose().ravel(), 0],
                        pair_predict[~mask.transpose().ravel(), 1],
                        'o',
                        color='indianred',
                        markersize=2,
                        label='test data')

    plt.text(-300, -300, f'epoch: {epoch:04d}', fontsize='medium')
    plt.xlabel("Ground Truth")
    plt.ylabel("Prediction")
    plt.xlim([0, max_groud_value])
    plt.ylim([0, max_groud_value])
    plt.legend(loc='upper left')
    plt.savefig(animation_path, format="png", dpi=200)
    plt.close()

    animation_count += 1


In [ ]:
# Initialize variables for early stopping
max_epoch = 5000
best_test_loss = float('inf')  # Set to infinity initially
patience_counter = 20
early_stopping_patience = 500  # Number of epochs with no improvement to wait before stopping
log_after_epochs = 30  # Start logging the model only after this number of epochs
animatiing_results = False

# Use GPU if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
data = data.to(device)

# Initialize Optimizer
optimizer_type = params["optimizer"]
optimizer_config = optimizer_type.value

if params["optimizer"] == OptimizerType.ADAM:
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=optimizer_config["learning_rate"],
                                 betas=optimizer_config["betas"],
                                 eps=optimizer_config["eps"],
                                 weight_decay=optimizer_config["weight_decay"])
    print(f'Setting up Adam optimizer with '
          f'lr: {optimizer_config["learning_rate"]}, '
          f'betas: {optimizer_config["betas"]}, '
          f'eps: {optimizer_config["eps"]}, '
          f'weight_decay: {optimizer_config["weight_decay"]}')
elif params["optimizer"] == OptimizerType.SGD:
    optimizer = torch.optim.SGD(model.parameters(),
                                lr=optimizer_config["learning_rate"],
                                momentum=optimizer_config["momentum"],
                                weight_decay=optimizer_config["weight_decay"])
    print(f'Setting up SGD optimizer with '
          f'lr: {optimizer_config["learning_rate"]}, '
          f'momentum: {optimizer_config["momentum"]}, '
          f'weight_decay: {optimizer_config["weight_decay"]}')

if params["scheduler"] == True:
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                           T_max=500,
                                                           eta_min=0.000001)
# Initialize lists to store losses
train_losses = []
test_losses = []

# Define loss function
criterion = MSELoss()

with mlflow.start_run() as run:

    mlflow.log_param("dataset", file_name)
    mlflow.log_param("num_params", count_parameters(model))
    # mlflow.log_artifact(fig_path)
    mlflow.log_artifact(mask_path / f"{file_name}_{time_tag}.npy")

    # Log the general parameters
    for key in params:
        if key != "optimizer":  # Exclude logging the optimizer enum object directly
            mlflow.log_param(key, params[key])

    # Log the optimizer type
    mlflow.log_param("optimizer_type", optimizer_type.name)

    # Log each parameter within the optimizer config
    for key, value in optimizer_config.items():
        mlflow.log_param(f"{optimizer_type.name.lower()}_{key}", value)
    
    progress_bar = tqdm(total=max_epoch, desc="Train", unit=" step", position=0, leave=True)

    try:
        for epoch in range(max_epoch):
            #train
            train_loss = np.sqrt(train())
            mlflow.log_metric(key="Train loss",
                              value=float(train_loss),
                              step=epoch)
            train_losses.append(train_loss)

            #test
            test_loss = torch.sqrt(test())
            mlflow.log_metric(key="Test loss",
                              value=float(test_loss),
                              step=epoch)
            test_losses.append(test_loss.item())

            # progress display
            progress_bar.update(1)
            progress_bar.set_postfix({
                "train loss": f"{train_loss:.2f}",
                "test loss": f"{test_loss:.2f}"
            })
            
            # Check if the validation loss has improved
            if test_loss < best_test_loss:
                best_test_loss = test_loss
                patience_counter = 0  # Reset patience counter
                # Only log the best model after `log_after_epochs` have passed
                if epoch >= log_after_epochs:
                    # if warnings of mismatching between the model's dependencies 
                    # and the current Python environment appear, 
                    # consider creating a suitable requirements.txt
                    mlflow.pytorch.log_model(model, "model", 
                                             pip_requirements="requirements_local.txt", 
                                             signature=SIGNATURE)  # Log the best model
                    print(f"Model logged at epoch {epoch} with test loss: {test_loss:.4f}")
            else:
                patience_counter += 1  # Increment patience counter
            
            # Early stopping condition
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch}. Best test loss: {best_test_loss:.4f}")
                break
            # scheduler.step()
            # if epoch % 10 == 0 and animatiing_results:
            if animatiing_results:
                create_animation(max_epoch)
                print("Plot one frame")

    finally:
        progress_bar.close()
        
        # Plotting the losses after training
        epochs = range(1, len(train_losses) + 1)
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
        
        # Plot for training loss
        ax1.plot(epochs, train_losses, 'b', label='Train Loss')
        ax1.set_title('Training Loss vs Epochs')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Loss')
        ax1.legend()
        
        # Plot for test loss
        ax2.plot(epochs, test_losses, 'r', label='Test Loss')
        ax2.set_title('Test Loss vs Epochs')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend()
        
        # Display the plots side by side
        plt.tight_layout()
        plt.show()
            
    # plot  
    result_path = Path().resolve(
    ) / "data" / "image" / f"{file_name}_{time_tag}.svg"
    result_Emap_path = Path().resolve(
    ) / "data" / "image" / f"{file_name}_{time_tag}_Emap.svg"
    
    prediction = model(data.x, data.edge_index)
    truth = data.y.detach().cpu().numpy()
    prediction = prediction.detach().cpu().numpy()
    error = prediction - truth
    pair_predict = np.concatenate((truth, prediction), axis=1)

    max_ground_value = max(data.y.detach().cpu().numpy())
    mask = data.train_mask.cpu().numpy()

    plt.figure(figsize=(4, 4))
    plt.plot(pair_predict[mask, 0],
             pair_predict[mask, 1],
             'o',
             color='royalblue',
             markersize=3,
             label='train data')
    plt.plot(pair_predict[~mask, 0],
             pair_predict[~mask, 1],
             'o',
             color='indianred',
             markersize=3,
             label='test data')

    # plt.plot(pair_predict[:, 0], pair_predict[:, 1], "o", markersize=3)
    # plt.plot(pair_predict[data.test_mask.cpu(), 0],
    #          pair_predict[data.test_mask.cpu(), 1],
    #          "or",
    #          markersize=3)
    plt.xlabel("Ground Truth")
    plt.ylabel("Prediction")
    plt.xlim([0, max_ground_value])
    plt.ylim([0, max_ground_value])
    plt.legend(loc='upper left')
    plt.savefig(result_path, format="svg")
    plt.show()

    lines = [
        LineString([node_loc[node[0]], node_loc[node[1]]])
        for node in lineGraph.nodes
    ]
    readout_gdf = gpd.GeoDataFrame({
        "error": error.ravel(),
        "geometry": lines
    },
                                   crs="EPSG:4839")

    fig, ax = plt.subplots(1, 1, figsize=(6, 4))
    readout_gdf.plot(column="error",
                     ax=ax,
                     legend=True,
                     legend_kwds={"shrink": 0.4},
                    #  norm=colors.LogNorm(vmin=1, vmax=5000)
                     )
    ax.set_axis_off()
    plt.savefig(result_Emap_path, format="svg")
    plt.show()

    mlflow.log_artifact(result_path)
    mlflow.log_artifact(result_Emap_path)

In [ ]:
# Initialize variables for early stopping
max_epoch = 5000
best_test_loss = float('inf')  # Set to infinity initially
patience_counter = 0
early_stopping_patience = 500  # Number of epochs with no improvement to wait before stopping
log_after_epochs = 20  # Start logging the model only after this number of epochs
animatiing_results = False

# Use GPU if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
data = data.to(device)

# Initialize Optimizer
optimizer_type = params["optimizer"]
optimizer_config = optimizer_type.value

if params["optimizer"] == OptimizerType.ADAM:
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=optimizer_config["learning_rate"],
                                 betas=optimizer_config["betas"],
                                 eps=optimizer_config["eps"],
                                 weight_decay=optimizer_config["weight_decay"])
    print(f'Setting up Adam optimizer with '
          f'lr: {optimizer_config["learning_rate"]}, '
          f'betas: {optimizer_config["betas"]}, '
          f'eps: {optimizer_config["eps"]}, '
          f'weight_decay: {optimizer_config["weight_decay"]}')
elif params["optimizer"] == OptimizerType.SGD:
    optimizer = torch.optim.SGD(model.parameters(),
                                lr=optimizer_config["learning_rate"],
                                momentum=optimizer_config["momentum"],
                                weight_decay=optimizer_config["weight_decay"])
    print(f'Setting up SGD optimizer with '
          f'lr: {optimizer_config["learning_rate"]}, '
          f'momentum: {optimizer_config["momentum"]}, '
          f'weight_decay: {optimizer_config["weight_decay"]}')

if params["scheduler"] == True:
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                           T_max=500,
                                                           eta_min=0.000001)
# Initialize lists to store losses
train_losses = []
test_losses = []

# Define loss function
criterion = MSELoss()

with mlflow.start_run() as run:

    mlflow.log_param("dataset", file_name)
    mlflow.log_param("num_params", count_parameters(model))
    # mlflow.log_artifact(fig_path)
    mlflow.log_artifact(mask_path / f"{file_name}_{time_tag}.npy")

    # Log the general parameters
    for key in params:
        if key != "optimizer":  # Exclude logging the optimizer enum object directly
            mlflow.log_param(key, params[key])

    # Log the optimizer type
    mlflow.log_param("optimizer_type", optimizer_type.name)

    # Log each parameter within the optimizer config
    for key, value in optimizer_config.items():
        mlflow.log_param(f"{optimizer_type.name.lower()}_{key}", value)
    
    progress_bar = tqdm(total=max_epoch, desc="Train", unit=" step", position=0, leave=True)

    try:
        for epoch in range(max_epoch):
            #train
            train_loss = np.sqrt(train())
            mlflow.log_metric(key="Train loss",
                              value=float(train_loss),
                              step=epoch)
            train_losses.append(train_loss)

            #test
            test_loss = torch.sqrt(test())
            mlflow.log_metric(key="Test loss",
                              value=float(test_loss),
                              step=epoch)
            test_losses.append(test_loss.item())

            # progress display
            progress_bar.update(1)
            progress_bar.set_postfix({
                "train loss": f"{train_loss:.2f}",
                "test loss": f"{test_loss:.2f}"
            })
            
            # Check if the validation loss has improved
            if test_loss < best_test_loss:
                best_test_loss = test_loss
                patience_counter = 0  # Reset patience counter
                # Only log the best model after `log_after_epochs` have passed
                if epoch >= log_after_epochs:
                    mlflow.pytorch.log_model(model, "model", 
                                             pip_requirements="requirements.txt", 
                                             signature=SIGNATURE)  # Log the best model
                    print(f"Model logged at epoch {epoch} with test loss: {test_loss:.4f}")
            else:
                patience_counter += 1  # Increment patience counter
            
            # Early stopping condition
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch}. Best test loss: {best_test_loss:.4f}")
                break
            # scheduler.step()
            # if epoch % 10 == 0 and animatiing_results:
            if animatiing_results:
                create_animation(max_epoch)
                print("Plot one frame")

    finally:
        progress_bar.close()
        
        # Plotting the losses after training
        epochs = range(1, len(train_losses) + 1)
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
        
        # Plot for training loss
        ax1.plot(epochs, train_losses, 'b', label='Train Loss')
        ax1.set_title('Training Loss vs Epochs')
        ax1.set_xlabel('Epochs')
        ax1.set_ylabel('Loss')
        ax1.legend()
        
        # Plot for test loss
        ax2.plot(epochs, test_losses, 'r', label='Test Loss')
        ax2.set_title('Test Loss vs Epochs')
        ax2.set_xlabel('Epochs')
        ax2.set_ylabel('Loss')
        ax2.legend()
        
        # Display the plots side by side
        plt.tight_layout()
        plt.show()
            
    # plot  
    result_path = Path().resolve(
    ) / "data" / "image" / f"{file_name}_{time_tag}.svg"
    result_Emap_path = Path().resolve(
    ) / "data" / "image" / f"{file_name}_{time_tag}_Emap.svg"
    
    prediction = model(data.x, data.edge_index)
    truth = data.y.detach().cpu().numpy()
    prediction = prediction.detach().cpu().numpy()
    error = prediction - truth
    pair_predict = np.concatenate((truth, prediction), axis=1)

    max_ground_value = max(data.y.detach().cpu().numpy())
    mask = data.train_mask.cpu().numpy()

    plt.figure(figsize=(4, 4))
    plt.plot(pair_predict[mask, 0],
             pair_predict[mask, 1],
             'o',
             color='royalblue',
             markersize=3,
             label='train data')
    plt.plot(pair_predict[~mask, 0],
             pair_predict[~mask, 1],
             'o',
             color='indianred',
             markersize=3,
             label='test data')

    # plt.plot(pair_predict[:, 0], pair_predict[:, 1], "o", markersize=3)
    # plt.plot(pair_predict[data.test_mask.cpu(), 0],
    #          pair_predict[data.test_mask.cpu(), 1],
    #          "or",
    #          markersize=3)
    plt.xlabel("Ground Truth")
    plt.ylabel("Prediction")
    plt.xlim([0, max_ground_value])
    plt.ylim([0, max_ground_value])
    plt.legend(loc='upper left')
    plt.savefig(result_path, format="svg")
    plt.show()

    lines = [
        LineString([node_loc[node[0]], node_loc[node[1]]])
        for node in lineGraph.nodes
    ]
    readout_gdf = gpd.GeoDataFrame({
        "error": error.ravel(),
        "geometry": lines
    },
                                   crs="EPSG:4839")

    fig, ax = plt.subplots(1, 1, figsize=(6, 4))
    readout_gdf.plot(column="error",
                     ax=ax,
                     legend=True,
                     legend_kwds={"shrink": 0.4},
                    #  norm=colors.LogNorm(vmin=1, vmax=5000)
                     )
    ax.set_axis_off()
    plt.savefig(result_Emap_path, format="svg")
    plt.show()

    mlflow.log_artifact(result_path)
    mlflow.log_artifact(result_Emap_path)

In [ ]:
n_bins = 50
bin_range1 = (0, np.max(truth))
bin_range2 = (0, np.max(prediction))
# bin_range2 = (0, 200)

fig, axes = plt.subplots(1, 2, figsize=(6, 3), sharey=True)
axes[0].hist(truth, bins=n_bins, range=bin_range1)
axes[0].set_title('Ground truth')
axes[0].set_xlim(left=0)

axes[1].hist(prediction, bins=n_bins, range=bin_range2)
axes[1].set_title('Prediction')
axes[1].set_xlim(left=0)

# plt.xlim(bin_range)
plt.tight_layout()
plt.show()

In [ ]:
# Define number of groups
num_groups = 5
max_value = 1000

pair_predict = np.concatenate((truth, prediction), axis=1)

# Split data into groups based on values of the first element
grouped_data = [
    pair_predict[(pair_predict[:, 0] >= i * max_value / num_groups)
                 & (pair_predict[:, 0] < (i + 1) * max_value / num_groups)]
    for i in range(num_groups)
]

# Plot histograms for each group
fig, axs = plt.subplots(num_groups, 1, figsize=(6, 12), sharex=True)
for i in range(num_groups):
    axs[i].hist(grouped_data[i][:, 1], bins=50)
    axs[i].set_title(
        f'Group {i+1}: {i*1000/num_groups} - {(i+1)*1000/num_groups}')

plt.show()


In [ ]:
plt.plot(pair_predict[:, 0], pair_predict[:, 1], "o", markersize=3)
plt.plot(pair_predict[data.test_mask, 0],
         pair_predict[data.test_mask, 1],
         "or",
         markersize=3)
plt.xlabel("Ground Truth")
plt.ylabel("Prediction")
plt.xlim([0, 200])
plt.ylim([0, 200])
plt.show()

In [ ]:
n_bins = 50
bin_range = (-400, 400)
plt.hist(error, bins=n_bins, range=bin_range)
plt.title('Prediction Error')
plt.xlim(bin_range)
plt.show()